## Import libraries

In [1]:
import ee
import geemap
import geemap.chart as ui_chart

## Create an interactive map

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [3]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      A2.5 Water Balance and Drought
#  Checkpoint:   A25c
#  Authors:      Ate Poortinga, Quyen Nguyen, Nyein Soe Thwal, Andréa Puzzi Nicolau
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Import the Lower Mekong boundary.
mekongBasin = ee.FeatureCollection(
'projects/gee-book/assets/A2-5/lowerMekongBasin')

# Center the map.
Map.centerObject(mekongBasin, 5)

# Add the Lower Mekong Basin boundary to the map.
Map.addLayer(mekongBasin, {}, 'Lower Mekong basin')

# Set start and end years.
startYear = 2010
endYear = 2020

# Create two date objects for start and end years.
startDate = ee.Date.fromYMD(startYear, 1, 1)
endDate = ee.Date.fromYMD(endYear + 1, 1, 1)

# Make a list with years.
years = ee.List.sequence(startYear, endYear)

# Make a list with months.
months = ee.List.sequence(1, 12)

# Import the CHIRPS dataset.
CHIRPS = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

# Filter for relevant time period.
CHIRPS = CHIRPS.filterDate(startDate, endDate)

# Import the MOD16 dataset.
mod16 = ee.ImageCollection('MODIS/006/MOD16A2').select('ET')

# Filter for relevant time period.
mod16 = mod16.filterDate(startDate, endDate)

# We apply a nested loop where we first iterate over
# the relevant years and then iterate over the relevant
# months. The function returns an image with P - ET
# for each month. A flatten is applied to convert an
# collection of collections into a single collection.
def func_kqp(y):
        def func_lkf(m):

            P = CHIRPS.filter(ee.Filter \
                    .calendarRange(y, y, 'year')) \
                .filter(ee.Filter.calendarRange(m, m,
                    'month')) \
                .sum()

            ET = mod16.filter(ee.Filter \
                    .calendarRange(y, y, 'year')) \
                .filter(ee.Filter.calendarRange(m, m,
                    'month')) \
                .sum() \
                .multiply(0.1)

            wb = P.subtract(ET).rename('wb')

            return wb.set('year', y) \
                .set('month', m) \
                .set('system:time_start', ee.Date \
                    .fromYMD(y, m, 1))

        return months.map(func_lkf)

waterBalance = ee.ImageCollection.fromImages(
    years.map(func_kqp).flatten()
)

# Add layer with monthly mean. note that we clip for the Mekong river basin.
balanceVis = {
    'min': -50,
    'max': 200,
    'palette': ['red', 'orange', 'yellow', 'blue', 'darkblue', 'purple']
}

Map.addLayer(waterBalance.mean().clip(mekongBasin),
    balanceVis,
    'Mean monthly water balance')

# Set the title and axis labels for the chart.
title = {
    'title': 'Monthly water balance',
    'hAxis': {
        'title': 'Time'
    },
    'vAxis': {
        'title': 'Evapotranspiration (mm)'
    },
    'colors': ['green']
}

# Plot the chart using the Mekong boundary.
chartMonthly = ui_chart.image_series_by_region(
image_collection = waterBalance,
regions = mekongBasin,
reducer = ee.Reducer.mean(),
band = 'wb',
scale = 500,
x_property = 'system:time_start',
series_property = None
)
chartMonthly.set_chart_type('ColumnChart')
#.setSeriesNames(['WB']) \
chartMonthly.set_options(options=title)
chartMonthly.display()


# -----------------------------------------------------------------------------------
# CHECKPOINT
# -----------------------------------------------------------------------------------

    geo       0_wb      100_wb      101_wb      102_wb      103_wb     104_wb  \
0  None -56.326106  170.126268  218.496329  332.494219  290.393544  184.52837   

     105_wb    106_wb     107_wb  ...       91_wb       92_wb       93_wb  \
0  22.79995 -48.64994 -36.153448  ...  314.170093  222.824368  106.393572   

       94_wb      95_wb      96_wb      97_wb      98_wb      99_wb  \
0 -15.667313 -45.240105 -47.158509 -49.684588 -14.291624  46.278311   

         9_wb  
0  131.049145  

[1 rows x 133 columns]


Exception: '[None] not in index'

## Display the interactive map

In [ ]:
Map